
* Github model source link : https://github.com/aub-mind/arabert

# AraELECTRA Model
* use the detector
* si un mot n'est pas similaire au mots générés par le modele ,alors il peut contenir une erreur

In [1]:
import pandas as pd  # for performance measure:
import gensim
import re  # regularexpression
import spacy
import nltk
import util as ut
from gensim import matutils  # utility fnc for pickling, common scipy operations etc
import numpy as np
import math
from transformers import pipeline
from arabert.preprocess import ArabertPreprocessor


In [2]:
MODEL_NAME = 'aubmindlab/Araelectra-base-generator'
fill_mask = pipeline(task="fill-mask",model=MODEL_NAME,tokenizer=MODEL_NAME)

In [3]:
#-- testing the method
text= "حقق منتخبنا الوطني الاول لكرة [MASK] كل اهدافه المطلوبة"
mask = "القسم"
words = fill_mask("حقق منتخبنا الوطني الاول لكرة [MASK] كل اهدافه المطلوبة")
#words = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
s=[]
t=[]
for e in words:
    t.append(e['token_str'])
    s.append(e['sequence'])
print("you wrote : ", mask)
print("did you mean : ",ut.lookslike(t,[mask]),"?")


you wrote :  القسم
did you mean :  القدم ?


In [4]:
def replace(mask, text2):
    text = text2
    text = text.split()
    text = list(map(lambda x: x.replace(mask, '[MASK]'), text))
    text = ' '.join(str(x) for x in text)
    return(text)


In [5]:
def araelectra(text):
    try:
        ok = True
        res = []
        print("your text = ", text, "\n")
        for mask in text.split():
            try:
                ok = True
                masked_text = ""
                masked_text = replace(mask, text)
                words = fill_mask(masked_text)
                s = []
                t = []
                for e in words[:-2]:
                    #        t.append(e['token_str'])
                    #print(e)
                    t.append(ut.clean_str(e['token_str']))
                    s.append(e['sequence'])
                # les mots parmi les mots générs qui ressemblent au mot à analyser
                correct = ut.lookslike(t, [ut.clean_str(mask)])
            #    correct = ut.lookslike(t, [mask])
            #    if correct != ut.clean_str(mask):
                if correct != mask and correct:
                    print("you wrote : [ ", mask,
                        "] did you mean : [ ", correct, "] ?")
                    ok = False
                    res.append((mask, correct))
            except : 
                continue
        return(res)
    except:
        return(res)


In [6]:
#testing ...
text = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
x= araelectra(text)
print("result = ",x)

your text =  حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة 

you wrote : [  حقق ] did you mean : [  وحقق ] ?
you wrote : [  القسم ] did you mean : [  القدم ] ?
result =  [('حقق', 'وحقق'), ('القسم', 'القدم')]


# Evaluation du modele 
*  si y_test n'apprtient pas à la liste générée res => loss

In [7]:
# parcours de la data 
# test 
#evaluation
df = pd.read_csv("../testdata.csv", dtype='string')
df


,sentence,wrong_word,correct_word,error_type
0,اصطاد الصياد السكة,السكة,السمكة,deletion
1,حقق المنتخب الوطني لكرة القسم كل اهدافه المطلوبة,القسم,القدم,substitution
2,تنطلق اليوم الدورة البرامجية الجديدة للتليفزيو...,الاشاعة,الاذاعة,substitution
3,طبخ الطباخ المركبات,المركبات,المرطبات,substitution
4,شهد السباق تفاعلا وحضورا كبيرا من محبي هذه الر...,العبرية,العربية,inversion error
5,أنا سعيد بسيناريو الفيلم الذي يلخص الواسع المصري,الواسع,الواقع,substitution
6,يقوم الدكتور بعلاج مرضى كثيرين يعانقون من امرا...,يعانقون,يعانون,adding
7,شارك الشباب في مهرجان الموسيقى الثقافي الحادي عرش,عرش,عشر,inversion
8,ارقى انواع الثقافات التي عرفتها الحضارة البري...,البرية,البشرية,deletion
9,اللام لا يصنع السعادة,اللام,المال,inversion


* y vs y_pred

In [8]:
def get_suggested_errors(y_pred) :
    s=[]
    for i in range(len(y_pred)):
        t=[]
        for j in range(len(y_pred[i])):
            t.append(y_pred[i][j][0])
        s.append(t)
    return(s)



In [9]:
test = [c for c in df.sentence]
y_araelectra_pred = [araelectra(c) for c in test]
y_pred = []
for i in y_araelectra_pred:
    if len(i) ==0 :
        y_pred.append([("0","0")])
    else:
        y_pred.append(i)
y = [y.lstrip().rstrip() for y in df.wrong_word]
y_pred = get_suggested_errors(y_pred)


your text =  اصطاد الصياد السكة 

your text =  حقق المنتخب الوطني لكرة القسم كل اهدافه المطلوبة 

you wrote : [  حقق ] did you mean : [  وحقق ] ?
you wrote : [  القسم ] did you mean : [  القدم ] ?
your text =  تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة التي تستمر طوال اشهر ابريل و مايو 

you wrote : [  الاشاعة ] did you mean : [  الاذاعة ] ?
you wrote : [  اشهر ] did you mean : [  شهر ] ?
your text =  طبخ الطباخ المركبات 

your text =  شهد السباق تفاعلا وحضورا كبيرا من محبي هذه الرياضة كونها تمثل رمزا للاصالة العبرية ونهجا على طريق الاجداد 

you wrote : [  شهد ] did you mean : [  وشهد ] ?
your text =  أنا سعيد بسيناريو الفيلم الذي يلخص الواسع المصري  

you wrote : [  سعيد ] did you mean : [  سعيدة ] ?
you wrote : [  الذي ] did you mean : [  والذي ] ?
you wrote : [  الواسع ] did you mean : [  الواقع ] ?
your text =  يقوم الدكتور بعلاج مرضى كثيرين يعانقون من امراض في الجهاز التنفسي  

you wrote : [  يعانقون ] did you mean : [  يعانون ] ?
your text =  شارك الشباب في مهرجان 

* precision 

In [10]:
precision = ut.precision2(y, y_pred)
print("Araelectra precision = ",precision)


Araelectra precision =  0.45454545454545453


# combinaison de modeles AraVec et AraELECTRA 
* aravec + aralecetra 
* calcul similarités (avec aravec) entres les mots suggérés par (araelectra)
*   araelectra suggere des couples (mot, erreur)
*   aravec retourne la meilleure correction 
*       similarité(منتخبنا, لمنتخبنا)>similarite(القسم,القدم)--> le mot erroné exact est "القسم"

In [11]:
#load Aravec model 
nlp = spacy.load("./spacy.aravec.model/")
# Define the preprocessing Class


class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = ut.clean_str(text)
        return self.tokenizer(preprocessed)


In [12]:
def combinedModels(text):
    try:
        a = araelectra(text)
        sim= [nlp(x).similarity(nlp(y)) for x,y in a]
        error = a[sim.index(min(sim))]
        print("\n the exact error is in word",error[0])
        return(error[0])
    except: 
        return("")

execution exemple

In [13]:
text = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"
x= combinedModels(text)


your text =  تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو 

you wrote : [  الاشاعة ] did you mean : [  الاذاعة ] ?
you wrote : [  برنامج ] did you mean : [  برامج ] ?
you wrote : [  التي ] did you mean : [  الذي ] ?
you wrote : [  اشهر ] did you mean : [  شهر ] ?

 the exact error is in word الاشاعة


C:\Users\PC\AppData\Local\Temp\ipykernel_15640\107581351.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim= [nlp(x).similarity(nlp(y)) for x,y in a]


## Evaluation de la combinaison

In [18]:
test = [c for c in df.sentence]
y = [y.lstrip().rstrip() for y in df.wrong_word]
y_combi_pred = [combinedModels(c) for c in test]


your text =  اصطاد الصياد السكة 

your text =  حقق المنتخب الوطني لكرة القسم كل اهدافه المطلوبة 

you wrote : [  حقق ] did you mean : [  وحقق ] ?
you wrote : [  القسم ] did you mean : [  القدم ] ?

 the exact error is in word القسم
your text =  تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة التي تستمر طوال اشهر ابريل و مايو 

you wrote : [  الاشاعة ] did you mean : [  الاذاعة ] ?
you wrote : [  اشهر ] did you mean : [  شهر ] ?

 the exact error is in word الاشاعة
your text =  طبخ الطباخ المركبات 



C:\Users\PC\AppData\Local\Temp\ipykernel_15640\107581351.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim= [nlp(x).similarity(nlp(y)) for x,y in a]


your text =  شهد السباق تفاعلا وحضورا كبيرا من محبي هذه الرياضة كونها تمثل رمزا للاصالة العبرية ونهجا على طريق الاجداد 

you wrote : [  شهد ] did you mean : [  وشهد ] ?

 the exact error is in word شهد
your text =  أنا سعيد بسيناريو الفيلم الذي يلخص الواسع المصري  

you wrote : [  سعيد ] did you mean : [  سعيدة ] ?
you wrote : [  الذي ] did you mean : [  والذي ] ?
you wrote : [  الواسع ] did you mean : [  الواقع ] ?

 the exact error is in word سعيد
your text =  يقوم الدكتور بعلاج مرضى كثيرين يعانقون من امراض في الجهاز التنفسي  

you wrote : [  يعانقون ] did you mean : [  يعانون ] ?

 the exact error is in word يعانقون
your text =  شارك الشباب في مهرجان الموسيقى الثقافي الحادي عرش 

you wrote : [  عرش ] did you mean : [  عشر ] ?

 the exact error is in word عرش
your text =   ارقى انواع الثقافات التي عرفتها الحضارة البرية على مر العصور  

you wrote : [  التي ] did you mean : [  والتي ] ?
you wrote : [  على ] did you mean : [  وعلي ] ?

 the exact error is in word على
your text =  اللام 

In [19]:
precisionn = ut.precision(y,y_combi_pred)
print("model combination precision = ",precisionn)

model combination precision =  0.36363636363636365
